In [1]:
import numpy as np
import pandas as pd
#import scipy.io as sio
import datetime as dt
import pickle

In [36]:
m1_df = pd.read_csv('processed_data/mayo-001.csv')

In [5]:
# drop severe BG readings that are not numeric
inds_to_drop = m1_df.loc[(m1_df['GlucoseValue_mg_dL_']=='Low') | (m1_df['GlucoseValue_mg_dL_']=='High') ].index.values # iloc does not like logical indexing
m1_df = m1_df.drop(inds_to_drop)


In [6]:
m1_df['BG'] = pd.to_numeric(m1_df['GlucoseValue_mg_dL_'])
m1_df = m1_df.drop(['GlucoseValue_mg_dL_'],axis='columns')

In [7]:
m1_df['Time'] = pd.to_datetime(m1_df['Time'])

In [9]:
#print(BG_PH)
#m1_df.loc[(m1_df['Time']>=t0-dt.timedelta(minutes=5)) & (m1_df['Time']<=t0)][['temp','step_mags','hr','eda']]

In [10]:
## this cell is now not in use, for testing only

# The big primary feature extraction

PH = dt.timedelta(minutes=30) # prediction horizon
margin = dt.timedelta(minutes=1) # margin of variation for the ph
window = dt.timedelta(minutes=2,seconds=30)
nos = 0
m1_feats = pd.DataFrame() # empty dataframe for input features 
#m1_labels = pd.DataFrame(columns=['BG_PH']) # labels to be predicted

for ind in m1_df.loc[~pd.isnull(m1_df['BG'])].index.values:
    #new_data = {}
    t0 = m1_df.loc[ind,'Time'].to_pydatetime() # time at which prediction is being made
    BG_PH = np.nanmean(m1_df.loc[(m1_df['Time']>=t0+PH-margin) & (m1_df['Time']<=t0+PH+margin)]['BG']) # get relevant BG vals for PH
    #if ~np.isnan(BG_PH): # if a BG 30 mins in future exists
    target_BG = np.nanmean(BG_PH)
    current_BG = m1_df.loc[ind,'BG']
    current_e4 = np.nanmean(m1_df.loc[(m1_df['Time']>=t0-dt.timedelta(minutes=5)) & (m1_df['Time']<=t0)][['temp','step_mags','hr','eda']],axis=0)
   # else:
    #nos = nos+1
    new_data = {t0':t0.hour,'BG0':current_BG,'temp0':current_e4[0],'step0':current_e4[1],'hr0':current_e4[2],'eda0':current_e4[3],'BG_PH':target_BG} # add basic features
    
    BG60s = m1_df.loc[(m1_df['Time']>=t0-dt.timedelta(minutes=60)) & (m1_df['Time']<=t0)][['Time','BG']]
    dBG60 = np.nanmax(BG60s['BG'])-current_BG # biggest BG drop past hour
    BG30s = BG60s.loc[(BG60s['Time']>=t0-dt.timedelta(minutes=30)) & (BG60s['Time']<=t0)]['BG']
    dBG30 = np.nanmax(BG60s['BG'])-current_BG # biggest BG drop past 30 mins
    BGday = m1_df.loc[(m1_df['Time']>=t0-dt.timedelta(hours=24)) & (m1_df['Time']<=t0)][['Time','BG']]
    BGday_var = np.var(BGday['BG']) # variance of BG over the past day
    new_data.update({'dBG60':dBG60,'dBG30':dBG30,'BGday_var':BGday_var}) # add BG change features
    
    
    m1_feats = m1_feats.append(new_data, ignore_index=True) # add the new entry to the dataset


C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: Mean of empty slice


In [1]:
#naninds = m1_df.loc[pd.isnull(m1_df['BG'])].index.values
#m1_df.loc[naninds,['BG']]
#print(m1_feats.describe())
#print(m1_feats.head())
#print(m1_feats.dtypes)



In [2]:
def load_process(filename):
    df = pd.read_csv(filename)
    # erroneuous variable name in some files
    if 'Var8' in df.columns:
        df = df.rename(columns={'Var8':'GlucoseValue_mg_dL_'})
    if ('Var4' in df.columns) & ~('Var5' in df.columns):
        df = df.rename(columns={'Var4':'GlucoseValue_mg_dL_'})
    if 'Var5' in df.columns:
        df = df.rename(columns={'Var4':'Time','Var5':'GlucoseValue_mg_dL_'}) # sans 9
    # drop severe BG readings that are not numeric
    inds_to_drop = df.loc[(df['GlucoseValue_mg_dL_']=='Low') | (df['GlucoseValue_mg_dL_']=='High') ].index.values # iloc does not like logical indexing
    df = df.drop(inds_to_drop)
    # change BG readings to numbers
    df['BG'] = pd.to_numeric(df['GlucoseValue_mg_dL_'])
    df = df.drop(['GlucoseValue_mg_dL_'],axis='columns')
    df['Time'] = pd.to_datetime(df['Time']) # cpnvert to datetime
    #print(df.describe())
    return df

In [3]:
def variability_feats(data, colnames, PH, margin):
    # generate features related to the variation of the data in the given columns for different periods of time
    # data is the "raw" data
    # colnames is a list of names of columsn in data to generate the features for
    # PH is the prediction horizon time
    # margin is the margin of error for the CGM timing
    
    data_feats = pd.DataFrame() # empty dataframe for input features 

    
    for ind in data.loc[~pd.isnull(data['BG'])].index.values: # where BG exists
        if ind % 500 ==0:
            print(ind/len(data.index)) # track progress
        print
        
        t0 = data.loc[ind,'Time'].to_pydatetime() # time at which prediction is being made
#        if clinic=='sansum':
#            t0= t0-dt.timedelta(hours=8) # adjust from UTC to California local time
#            print('Sans')
#        if clinic=='mayo':
#            t0 = t0-dt.timedelta()
        # get relevant BG vals for PH
        BG_PH = np.nanmean(data.loc[(data['Time']>=t0+PH-margin) & (data['Time']<=t0+PH+margin)]['BG']) 
        target_BG = np.nanmean(BG_PH)
        current_BG = data.loc[ind,'BG']
        current_e4 = np.nanmean(data.loc[(data['Time']>=t0-dt.timedelta(minutes=5)) & 
                                         (data['Time']<=t0)][['temp','step_mags','hr','eda']],axis=0)

        new_data = {'t0':t0.hour,'BG0':current_BG,'temp0':current_e4[0],'step0':current_e4[1],
                    'hr0':current_e4[2],'eda0':current_e4[3],'BG_PH':target_BG,'full_time':t0} # add basic features

        
        for colname in colnames:
            current_data = new_data['{}0'.format(colname)] # current reading of relevant column

            ## THESE CURRENTLY RETURN SOMETHING EVEN WITH SIGNIFICANT MISSING DATA, CONSIDER CHANGING THIS
            data60s = data.loc[(data['Time']>=t0-dt.timedelta(minutes=60)) & (data['Time']<=t0)][['Time',colname]]
            d60 = np.nanmax(data60s[colname])-current_data # biggest data drop past hour
            data30s = data60s.loc[(data60s['Time']>=t0-dt.timedelta(minutes=30)) & (data60s['Time']<=t0)][['Time',colname]]
            d30 = np.nanmax(data30s[colname])-current_data # biggest data drop past 30 mins
            dataday = data.loc[(data['Time']>=t0-dt.timedelta(hours=24)) & (data['Time']<=t0)][['Time',colname]]
            day_var = np.var(dataday[colname]) # variance of data over the past day
            new_data.update({'d{}60'.format(colname):d60,'d{}30'.format(colname):d30,
                             '{}day_var'.format(colname):day_var}) # add data change features

        data_feats = data_feats.append(new_data, ignore_index=True) # add the new entry to the dataset
        
    return data_feats


In [4]:
def step_feats(data, colname):
    
    for ind in data.index.values:
        t0 = data.loc[ind,'full_time'] # time at which prediction is being made
        
        # mean step data in each 1 hr window 
        data.loc[ind,'{}mean_1hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=60)) & (data['full_time']<=t0)][colname])
        data.loc[ind,'{}mean_2hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=120)) & (data['full_time']<=t0-dt.timedelta(minutes=60))][colname])
        data.loc[ind,'{}mean_3hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=180)) & (data['full_time']<=t0-dt.timedelta(minutes=120))][colname])
        data.loc[ind,'{}mean_4hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=240)) & (data['full_time']<=t0-dt.timedelta(minutes=180))][colname])
        data.loc[ind,'{}mean_5hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=300)) & (data['full_time']<=t0-dt.timedelta(minutes=240))][colname])

    return data
        

In [5]:
def HR_feats(data, colname):
    
    for ind in data.index.values:
        t0 = data.loc[ind,'full_time'] # time at which prediction is being made
        
        daymean_hr = data.loc[(data['full_time']>=t0-dt.timedelta(hours=24)) & (data['full_time']<=t0)][colname].values[0]
        
        
        data.loc[ind, '{}adj_curr'.format(colname)] = np.nanmean(data.loc[data['full_time']==t0][colname]/daymean_hr)
        
        # mean adjusted HR data in each 1 hr window 
        data.loc[ind,'{}adj_mean_1hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=60))
                                                                             & (data['full_time']<=t0)][colname])/daymean_hr
        data.loc[ind,'{}adj_mean_2hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=120)) 
                                                                             & (data['full_time']<=t0-dt.timedelta(minutes=60))][colname])/daymean_hr
        data.loc[ind,'{}adj_mean_3hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=180)) 
                                                                             & (data['full_time']<=t0-dt.timedelta(minutes=120))][colname])/daymean_hr
        data.loc[ind,'{}adj_mean_4hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=240)) 
                                                                             & (data['full_time']<=t0-dt.timedelta(minutes=180))][colname])/daymean_hr
        data.loc[ind,'{}adj_mean_5hr'.format(colname)] = np.nanmean(data.loc[(data['full_time']>=t0-dt.timedelta(minutes=300)) 
                                                                             & (data['full_time']<=t0-dt.timedelta(minutes=240))][colname])/daymean_hr

    return data
        

In [13]:
def feature_extract(data):

    # The big primary feature extraction

    PH = dt.timedelta(minutes=30) # prediction horizon
    margin = dt.timedelta(minutes=1) # margin of variation for the ph
    #window = dt.timedelta(minutes=2,seconds=30)
    #m1_labels = pd.DataFrame(columns=['BG_PH']) # labels to be predicted
    
    variability_columns = ['BG','temp','eda']
    data_feats = variability_feats(data, variability_columns, PH, margin)
    
    data_feats = handle_missing(data_feats)
    print('var feats')
    print(data_feats.head())
    data_feats = step_feats(data_feats, 'step0')
    print('step feats')
    print(data_feats.head())
    data_feats = HR_feats(data_feats,'hr0')
    print(data_feats.columns)
    
    #data_feats = data_feats.drop(columns=['full_time']) # drop full time string now that features have been generated
    

    return data_feats

In [14]:
def handle_missing(data):
    # drop all entries missing a label or e4 data, MAY WANT TO CHANGE THIS LATER BUT IDK
    data = data.dropna(axis=0, subset=['BG_PH','temp0','step0','hr0','eda0'])
    return data

In [15]:
#test = load_process('processed_data/mayo-001.csv')
#test = feature_extract(test)


In [16]:
#test.columns

In [17]:
# patient suffixes for file names
mayo_suffs =  ['001','002','003','004','005','006','007','008']
sans_suffs = ['01','02','03','04','05','06','07','08','09','10']

In [18]:
for suff in mayo_suffs:
    print('patient', suff)
    filename = 'processed_data/mayo-%s.csv' % suff
    patient_features = feature_extract(load_process(filename))
    print(patient_features.columns)
    #patient_features = patient_features.drop('full_time')
    fileout = 'feature_data/mayo-%s.p' % suff
    pickle.dump(patient_features, open(fileout,'wb'))
    

patient 001
0.0


C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: All-NaN axis encountered
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: All-NaN axis encountered


0.02864672854360032
0.08594018563080097
0.1432336427180016
0.17188037126160194
0.20052709980520225
0.22917382834880257
0.2864672854360032
0.37240747106680416
0.4010541996104045
0.4297009281540048
0.45834765669760513
0.5156411137848058
0.5442878423284061
0.5729345708720064
0.6015812994156068
0.6302280279592071
0.6588747565028074
0.6875214850464078
0.7734616706772086
0.802108399220809
0.88804858485161
0.9453420419388106
0.973988770482411
var feats
      BG0  BG_PH   BGday_var  dBG30  dBG60  deda30  deda60   dtemp30  \
59  151.0  182.0  777.760833    0.0    0.0     0.0     0.0  0.000000   
60  156.0  187.0  765.249664    0.0    0.0     0.0     0.0  0.162394   
61  159.0  184.0  753.637877    0.0    0.0     0.0     0.0  0.335728   
62  170.0  189.0  746.556815    0.0    0.0     0.0     0.0  0.351261   
63  179.0  193.0  745.609131    0.0    0.0     0.0     0.0  0.414194   

     dtemp60      eda0  edaday_var           full_time        hr0     step0  \
59  0.000000  0.049464    0.000000 201

C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


step feats
      BG0  BG_PH   BGday_var  dBG30  dBG60  deda30  deda60   dtemp30  \
59  151.0  182.0  777.760833    0.0    0.0     0.0     0.0  0.000000   
60  156.0  187.0  765.249664    0.0    0.0     0.0     0.0  0.162394   
61  159.0  184.0  753.637877    0.0    0.0     0.0     0.0  0.335728   
62  170.0  189.0  746.556815    0.0    0.0     0.0     0.0  0.351261   
63  179.0  193.0  745.609131    0.0    0.0     0.0     0.0  0.414194   

     dtemp60      eda0  ...        hr0     step0    t0      temp0  \
59  0.000000  0.049464  ...  67.852683  0.181963  15.0  31.193261   
60  0.162394  0.207864  ...  69.369600  0.848015  15.0  31.030867   
61  0.335728  0.235614  ...  67.396800  0.987833  15.0  30.857533   
62  0.351261  0.246916  ...  79.461400  2.401050  15.0  30.842000   
63  0.414194  0.293356  ...  93.103267  6.323290  15.0  30.779067   

    tempday_var  step0mean_1hr  step0mean_2hr  step0mean_3hr  step0mean_4hr  \
59     0.000000       0.181963            NaN            NaN  

C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice


Index(['BG0', 'BG_PH', 'BGday_var', 'dBG30', 'dBG60', 'deda30', 'deda60',
       'dtemp30', 'dtemp60', 'eda0', 'edaday_var', 'full_time', 'hr0', 'step0',
       't0', 'temp0', 'tempday_var', 'step0mean_1hr', 'step0mean_2hr',
       'step0mean_3hr', 'step0mean_4hr', 'step0mean_5hr', 'hr0adj_curr',
       'hr0adj_mean_1hr', 'hr0adj_mean_2hr', 'hr0adj_mean_3hr',
       'hr0adj_mean_4hr', 'hr0adj_mean_5hr'],
      dtype='object')
Index(['BG0', 'BG_PH', 'BGday_var', 'dBG30', 'dBG60', 'deda30', 'deda60',
       'dtemp30', 'dtemp60', 'eda0', 'edaday_var', 'full_time', 'hr0', 'step0',
       't0', 'temp0', 'tempday_var', 'step0mean_1hr', 'step0mean_2hr',
       'step0mean_3hr', 'step0mean_4hr', 'step0mean_5hr', 'hr0adj_curr',
       'hr0adj_mean_1hr', 'hr0adj_mean_2hr', 'hr0adj_mean_3hr',
       'hr0adj_mean_4hr', 'hr0adj_mean_5hr'],
      dtype='object')
patient 002
0.0
0.025200342724661054
0.05040068544932211
0.10080137089864422
0.15120205634796632
0.1764023990726274
0.2268030845219495
0.252

In [19]:
for suff in sans_suffs:
    print('patient', suff)
    filename = 'processed_data/sansum-%s.csv' % suff
    patient_features = handle_missing(feature_extract(load_process(filename)))
    fileout = 'feature_data/sansum-%s.p' % suff
    pickle.dump(patient_features, open(fileout,'wb'))

patient 01


C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


0.0


C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: All-NaN axis encountered
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: All-NaN axis encountered
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: Mean of empty slice


0.01605651894669236
0.03211303789338472
0.04816955684007707
0.06422607578676943
0.08028259473346179
0.09633911368015415
0.1123956326268465
0.12845215157353887
0.14450867052023122
0.16056518946692358
0.17662170841361594
0.1926782273603083
0.20873474630700065
0.224791265253693
0.24084778420038536
0.25690430314707774
0.2729608220937701
0.28901734104046245
0.3050738599871548
0.32113037893384716
0.3371868978805395
0.35324341682723187
0.3692999357739242
0.3853564547206166
0.40141297366730894
0.4174694926140013
0.43352601156069365
0.449582530507386
0.46563904945407836
0.4816955684007707
0.49775208734746307
0.5138086062941555
0.5298651252408478
0.5619781631342325
0.5780346820809249
0.6101477199743096
0.626204238921002
0.6422607578676943
0.6583172768143867
0.674373795761079
0.7064868336544637
0.7546563904945408
0.7707129094412332
0.8028259473346179
0.8349389852280026
0.8509955041746949
0.8670520231213873
0.8831085420680796
0.899165061014772
0.9312780989081567
0.9473346178548491
0.97944765574823

C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


step feats
         BG0  BG_PH   BGday_var  dBG30  dBG60    deda30    deda60   dtemp30  \
14532  231.0  225.0  768.204895    2.0    2.0  0.000000  0.000000  0.000000   
14533  233.0  222.0  791.986767    0.0    0.0  0.000000  0.000000  0.107836   
14534  233.0  220.0  815.692670    0.0    0.0  0.000000  0.000000  0.342148   
14535  233.0  218.0  839.264010    0.0    0.0  1.291043  1.291043  0.283903   
14536  231.0  215.0  861.580636    2.0    2.0  2.819058  2.819058  0.240103   

        dtemp60      eda0  ...         hr0      step0    t0      temp0  \
14532  0.000000  4.068016  ...   82.277383   5.180011  21.0  25.207436   
14533  0.107836  7.713070  ...  109.565033  15.248721  21.0  25.099600   
14534  0.342148  7.876621  ...   90.469143   5.764094  21.0  24.865288   
14535  0.283903  6.585577  ...  119.765267   3.991873  21.0  24.923533   
14536  0.240103  5.057563  ...   88.225000   4.531806  22.0  24.967333   

       tempday_var  step0mean_1hr  step0mean_2hr  step0mean_3hr  \
14

C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: Mean of empty slice
C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice


Index(['BG0', 'BG_PH', 'BGday_var', 'dBG30', 'dBG60', 'deda30', 'deda60',
       'dtemp30', 'dtemp60', 'eda0', 'edaday_var', 'full_time', 'hr0', 'step0',
       't0', 'temp0', 'tempday_var', 'step0mean_1hr', 'step0mean_2hr',
       'step0mean_3hr', 'step0mean_4hr', 'step0mean_5hr', 'hr0adj_curr',
       'hr0adj_mean_1hr', 'hr0adj_mean_2hr', 'hr0adj_mean_3hr',
       'hr0adj_mean_4hr', 'hr0adj_mean_5hr'],
      dtype='object')
patient 02
0.10540369615627855
0.14053826154170473
0.2108073923125571
0.24594195769798327
0.28107652308340947
0.31621108846883567
0.386480219239688
0.4216147846251142
0.45674935001054034
0.5621530461668189
0.5972876115522451
0.6675567423230975
0.7026913077085236
0.7378258730939499
0.772960438479376
0.8080950038648022
0.8432295692502284
0.8783641346356545
0.9486332654065069
var feats
     BG0  BG_PH   BGday_var  dBG30  dBG60  deda30  deda60  dtemp30  dtemp60  \
88  86.0  105.0  574.814291    9.0   19.0     0.0     0.0      0.0      0.0   
89  85.0  110.0  584.99555

In [72]:
df = pd.read_csv(filename)
print(df.describe())
# drop severe BG readings that are not numeric
inds_to_drop = df.loc[(df['GlucoseValue_mg_dL_']=='Low') | (df['GlucoseValue_mg_dL_']=='High') ].index.values # iloc does not like logical indexing
df = df.drop(inds_to_drop)
# change BG readings to numbers
df['BG'] = pd.to_numeric(df['GlucoseValue_mg_dL_'])
df = df.drop(['GlucoseValue_mg_dL_'],axis='columns')
df['Time'] = pd.to_datetime(df['Time']) # cpnvert to datetime

              Var5         temp    step_mags           hr          eda
count  8413.000000  8315.000000  8315.000000  8311.000000  8315.000000
mean    119.951385    33.928059     1.942868    89.250449     0.913320
std      43.787141     2.114177     2.556277    14.241604     1.473155
min      40.000000    22.203800     0.006096    63.720400     0.000000
25%      89.000000    33.592900     0.284790    79.576417     0.243244
50%     111.000000    34.475100     1.117753    87.379467     0.421293
75%     143.000000    35.112983     2.459875    95.620567     1.108704
max     357.000000    36.535600    21.725415   191.133333    44.792798


KeyError: 'GlucoseValue_mg_dL_'

In [31]:
np.nanmean([])

C:\Users\coh275\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


nan

    for ind in data.loc[~pd.isnull(data['BG'])].index.values:
        if ind % 500 ==0:
            print(ind/len(data.index))
        
        t0 = data.loc[ind,'Time'].to_pydatetime() # time at which prediction is being made
        BG_PH = np.nanmean(data.loc[(data['Time']>=t0+PH-margin) & (data['Time']<=t0+PH+margin)]['BG']) # get relevant BG vals for PH
        #if ~np.isnan(BG_PH): # if a BG 30 mins in future exists
        target_BG = np.nanmean(BG_PH)
        current_BG = data.loc[ind,'BG']
        current_e4 = np.nanmean(data.loc[(data['Time']>=t0-dt.timedelta(minutes=5)) & (data['Time']<=t0)][['temp','step_mags','hr','eda']],axis=0)

        new_data = {'t0':t0.hour,'BG0':current_BG,'temp0':current_e4[0],'step0':current_e4[1],'hr0':current_e4[2],'eda0':current_e4[3],'BG_PH':target_BG} # add basic features

        ## THESE CURRENTLY RETURN SOMETHING EVEN WITH SIGNIFICANT MISSING DATA, CONSIDER CHANGING THIS
        BG60s = data.loc[(data['Time']>=t0-dt.timedelta(minutes=60)) & (data['Time']<=t0)][['Time','BG']]
        dBG60 = np.nanmax(BG60s['BG'])-current_BG # biggest BG drop past hour
        BG30s = BG60s.loc[(BG60s['Time']>=t0-dt.timedelta(minutes=30)) & (BG60s['Time']<=t0)]['BG']
        dBG30 = np.nanmax(BG60s['BG'])-current_BG # biggest BG drop past 30 mins
        BGday = data.loc[(data['Time']>=t0-dt.timedelta(hours=24)) & (data['Time']<=t0)][['Time','BG']]
        BGday_var = np.var(BGday['BG']) # variance of BG over the past day
        new_data.update({'dBG60':dBG60,'dBG30':dBG30,'BGday_var':BGday_var}) # add BG change features

        data_feats = data_feats.append(new_data, ignore_index=True) # add the new entry to the dataset
        
        
Old code, no longer used